In [1]:
import pandas as pd
import numpy as np
import datetime


from sklearn.ensemble import RandomForestClassifier

In [60]:
input_fire_csv_path = '../data/FW_Veg_Rem_Combined.csv'
input_city_csv_path =  '../data/city_attributes.csv'
input_humidity_csv_path = '../data/humidity.csv'
input_pressure_csv_path = '../data/pressure.csv'
input_temperature_csv_path = '../data/temperature.csv'
input_windD_csv_path = '../data/wind_direction.csv'
input_windS_csv_path =  '../data/wind_speed.csv'

In [61]:
df_fire = pd.read_csv(input_fire_csv_path)
df_city = pd.read_csv(input_city_csv_path)
df_hum = pd.read_csv(input_humidity_csv_path)
df_pre = pd.read_csv(input_pressure_csv_path)
df_tem = pd.read_csv(input_temperature_csv_path)
df_windD = pd.read_csv(input_windD_csv_path)
df_windS = pd.read_csv(input_windS_csv_path)

In [62]:
df_dic = {  'humidity':df_hum,
            'pressure':df_pre,
            'temperature':df_tem,
            'wind_direction':df_windD,
            'wind_speed':df_windS}

In [63]:
for key in df_dic.keys():
    df_dic[key]['datetime'] = pd.to_datetime(df_dic[key]['datetime']).apply(datetime.datetime.date)
    df_dic[key] = df_dic[key].groupby('datetime').mean()
    df_dic[key] = df_dic[key].stack(0).reset_index()
    df_dic[key].rename(columns={'datetime':'datetime','level_1':'city', 0:key}, inplace=True)
df_hum, df_pre, df_tem, df_windD, df_windS = df_dic.values()

In [64]:
df_final = df_hum
for key in df_dic.keys():
    if key == 'humidity':
        continue
    else:
        df_final=df_final.merge(df_dic[key], on=['datetime', 'city'])
df_final

,datetime,city,humidity,pressure,temperature,wind_direction,wind_speed
0,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000
1,2012-10-01,San Francisco,83.000000,1009.727273,289.416642,122.363636,1.636364
2,2012-10-01,Seattle,78.000000,1030.000000,281.767262,32.272727,0.000000
3,2012-10-01,Los Angeles,88.000000,1013.000000,291.846501,0.000000,0.000000
4,2012-10-01,San Diego,79.909091,1013.000000,291.573495,0.000000,0.000000
...,...,...,...,...,...,...,...
67533,2017-11-30,Pittsburgh,42.000000,1025.000000,279.190000,360.000000,2.000000
67534,2017-11-30,Toronto,60.000000,1027.000000,274.510000,330.000000,3.000000
67535,2017-11-30,Philadelphia,32.000000,1024.000000,283.420000,360.000000,4.000000
67536,2017-11-30,Montreal,58.000000,1027.000000,271.800000,300.000000,4.000000


In [68]:
df_city.rename(columns={'City':'city'}, inplace=True)

In [69]:
df_final.merge(df_city, on=['city'])

,datetime,city,humidity,pressure,temperature,wind_direction,wind_speed,Country,Latitude,Longitude
0,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000,United States,45.523449,-122.676208
1,2012-10-02,Portland,65.833333,1023.583333,286.137728,214.041667,1.291667,United States,45.523449,-122.676208
2,2012-10-03,Portland,66.208333,1021.083333,289.599792,228.333333,2.625000,United States,45.523449,-122.676208
3,2012-10-04,Portland,51.166667,1022.875000,286.482500,206.750000,4.625000,United States,45.523449,-122.676208
4,2012-10-05,Portland,40.391304,1022.916667,288.286042,182.250000,3.708333,United States,45.523449,-122.676208
...,...,...,...,...,...,...,...,...,...,...
67533,2017-10-24,Vancouver,84.208333,1035.458333,283.835417,139.333333,1.708333,Canada,49.249660,-123.119339
67534,2017-10-25,Vancouver,92.583333,1021.166667,283.138333,156.583333,2.666667,Canada,49.249660,-123.119339
67535,2017-10-26,Vancouver,93.208333,1024.166667,283.408333,310.958333,3.166667,Canada,49.249660,-123.119339
67536,2017-10-27,Vancouver,93.583333,1024.291667,283.085417,281.291667,2.125000,Canada,49.249660,-123.119339


In [6]:
fire_df = veg_rem_combined_df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', ])
fire_df.head()

In [54]:
humidity_df = pd.read_csv(r'../data/humidity.csv')

In [55]:
humidity_df.head(2)

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN
1,2012-10-01 13:00:00,76.0,81.0,88.0,81.0,88.0,82.0,22.0,23.0,50.0,...,71.0,58.0,93.0,68.0,50.0,63.0,22.0,51.0,51.0,50.0


In [56]:
humidity_df['datetime'] = pd.to_datetime(humidity_df['datetime']).apply(datetime.datetime.date)
humidity_df = humidity_df.groupby('datetime').mean()

In [57]:
humidity_df

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
datetime,,,,,,,,,,,,,,,,,,,,,
2012-10-01,78.272727,78.727273,83.000000,78.000000,88.000000,79.909091,20.727273,24.181818,48.909091,63.181818,...,68.272727,56.272727,75.727273,68.000000,54.818182,62.909091,23.250000,51.000,51.00,50.000000
2012-10-02,77.375000,65.833333,65.958333,65.166667,64.916667,65.875000,16.166667,24.476190,40.363636,66.647059,...,54.500000,50.950000,39.500000,68.750000,72.916667,69.708333,33.833333,53.500,53.50,52.750000
2012-10-03,71.625000,66.208333,47.083333,58.416667,44.875000,59.750000,14.500000,18.380952,31.904762,36.869565,...,78.791667,68.541667,88.400000,86.041667,75.541667,77.208333,47.291667,69.375,69.75,62.291667
2012-10-04,49.000000,51.166667,60.166667,48.869565,66.625000,75.583333,15.166667,20.238095,19.681818,14.750000,...,88.565217,94.545455,94.750000,84.750000,67.458333,64.625000,49.333333,71.250,71.25,63.375000
2012-10-05,43.409091,40.391304,69.666667,48.521739,70.250000,72.791667,14.647059,27.777778,18.250000,49.500000,...,84.750000,93.041667,88.000000,73.333333,65.666667,63.416667,46.875000,70.500,70.50,67.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-26,NaN,87.625000,NaN,93.291667,86.666667,88.250000,38.166667,43.208333,29.750000,47.833333,...,53.125000,NaN,78.166667,54.666667,NaN,NaN,NaN,NaN,NaN,NaN
2017-11-27,NaN,92.875000,NaN,85.416667,77.333333,81.333333,34.458333,34.041667,29.041667,25.958333,...,49.375000,NaN,74.125000,55.958333,NaN,NaN,NaN,NaN,NaN,NaN
2017-11-28,NaN,88.458333,NaN,85.750000,54.958333,76.375000,33.125000,46.208333,31.458333,47.666667,...,54.666667,NaN,64.833333,57.541667,NaN,NaN,NaN,NaN,NaN,NaN
